In [1]:
import numpy as np
import scipy as sci
import bisect

In [2]:
# Noisy Range Count for DP element count within a given range
def NoisyRC_normal(left, right, min_val, max_val, rho, D):
    left_pos = bisect.bisect_left(D, left)
    right_pos = bisect.bisect_left(D, right)
    return right_pos - left_pos + 1 + np.random.normal(0, np.sqrt(np.log2(max_val - min_val) / (2 * rho)), 1)[0]

# Noisy Range Count for DP element count within a given range
def NoisyRC_normal_float(left, right, min_val, max_val, rho, D, precision = 1e-4):
    left_pos = bisect.bisect_left(D, left)
    right_pos = bisect.bisect_left(D, right)
    return right_pos - left_pos + 1 + np.random.normal(0, np.sqrt(np.log2((max_val - min_val) / precision) / (2 * rho)), 1)[0]

In [3]:
# Noisy Range Count for DP element count within a given range
def NoisyRC_laplace(left, right, min_val, max_val, epsilon, D):
    left_pos = bisect.bisect_left(D, left)
    right_pos = bisect.bisect_left(D, right)
    return right_pos - left_pos + 1 + np.random.laplace(0, np.log2(max_val - min_val) / epsilon, 1)[0]

# Noisy Range Count for DP element count within a given range
def NoisyRC_laplace_float(left, right, min_val, max_val, epsilon, D, precision = 1e-4):
    left_pos = bisect.bisect_left(D, left)
    right_pos = bisect.bisect_left(D, right)
    return right_pos - left_pos + 1 + np.random.laplace(0, np.log2((max_val - min_val) / precision) / epsilon, 1)[0]

In [4]:
# PrivQuant for DP Quantile Selection by Binary Search
# D: 1-d vector
# m: desired m-th quantile of D
# rho: expected rho-CDP
def PrivQuant_normal(D, min_val, max_val, m, rho):
    D = np.sort(D)
    left = min_val
    right = max_val
    while left < right:
        mid = int(np.floor((left + right) / 2))
        c_hat = NoisyRC_normal(0, mid, min_val, max_val, rho, D)
        if(c_hat <= m):
            left = mid + 1
        else:
            right = mid
    return int(np.floor((left + right) / 2))

In [5]:
# PrivQuant for DP Quantile Selection by Binary Search
# D: 1-d vector
# m: desired m-th quantile of D
# rho: expected rho-CDP
def PrivQuant_laplace(D, min_val, max_val, m, epsilon):
    D = np.sort(D)
    left = min_val
    right = max_val
    while left < right:
        mid = int(np.floor((left + right) / 2))
        c_hat = NoisyRC_laplace(0, mid, min_val, max_val, epsilon, D)
        if(c_hat <= m):
            left = mid + 1
        else:
            right = mid
    return int(np.floor((left + right) / 2))

In [6]:
# PrivQuant for DP Quantile Selection by Binary Search
# D: 1-d vector
# m: desired m-th quantile of D
# rho: expected rho-CDP
def PrivQuant_normal_float(D, min_val, max_val, m, rho, precision = 1e-4):
    D = np.sort(D)
    left = min_val
    right = max_val
    while (right - left) > precision:
        mid = (left + right) / 2
        c_hat = NoisyRC_normal(0, mid, rho, D)
        if(c_hat <= m):
            left = mid + precision
        else:
            right = mid
    return (left + right) / 2

In [7]:
# PrivQuant for DP Quantile Selection by Binary Search
# D: 1-d vector
# m: desired m-th quantile of D
# rho: expected rho-CDP
def PrivQuant_laplace_float(D, min_val, max_val, m, epsilon, precision = 1e-4):
    D = np.sort(D)
    left = min_val
    right = max_val
    while (right - left) > epsilon:
        mid = (left + right) / 2
        c_hat = NoisyRC_normal(0, mid, precision, D)
        if(c_hat <= m):
            left = mid + epsilon
        else:
            right = mid
    return (left + right) / 2

In [8]:
# Assume that input D is sorted in ascending order
def GetRankError_normal(min_val, max_val, d, rho, beta):
    u = max_val - min_val
    n = np.log2(d * u)
    simga_squared = n / (2 * rho)
    return np.sqrt(2 * simga_squared * np.log(n / beta))

def GetRankError_normal_float(min_val, max_val, d, rho, beta, precision = 1e-4):
    u = max_val - min_val
    n = np.log2(d * u) / precision
    simga_squared = n / (2 * rho)
    return np.sqrt(2 * simga_squared * np.log(n / beta))

def GetRankError_laplace(min_val, max_val, d, epsilon, beta):
    u = max_val - min_val
    n = np.log2(d * u)
    b = n / epsilon
    return -b * np.log(1 - np.float_power(1 - beta, 1 / n))

def GetRankError_laplace_float(min_val, max_val, d, epsilon, beta, precision = 1e-4):
    u = max_val - min_val
    n = np.log2(d * u) / precision
    b = n / epsilon
    return -b * np.log(1 - np.float_power(1 - beta, 1 / n))

def GetRankError_laplace_relaxed(min_val, max_val, d, epsilon, beta):
    u = max_val - min_val
    n = np.log2(d * u)
    b = n / epsilon
    return -b * np.log(beta / n)

def GetRankError_laplace_relaxed_float(min_val, max_val, d, epsilon, beta, precision = 1e-4):
    u = max_val - min_val
    n = np.log2(d * u) / precision
    b = n / epsilon
    return -b * np.log(beta / n)

In [9]:
def GetOptimalQuantile_normal(n, d, tau, rho):
    return max(n - max(np.sqrt(2 * d / rho), tau), 1)

def GetOptimalQuantile_laplace(n, d, tau, epsilon):
    return max(n - max(np.sqrt(8 * d) / epsilon, tau), 1)

### Packing entire thing together

In [10]:
def ClippedSumEstimator_normal(D, min_val, max_val, rho, beta):
    # Calculate rank error tau
    tau = GetRankError_normal(min_val, max_val, 1, rho / 4, beta)

    # Get the optimal clipped quantile m
    # Note that we should use 1/4 of rho here
    m = GetOptimalQuantile_normal(D.size, 1, tau, rho / 4)

    # Get clipping threshold C
    clip_thresh = PrivQuant_normal(D, min_val, max_val, m, rho / 4)

    # Calculate clipped value of D
    D_clipped = np.clip(D, a_min = 0, a_max = clip_thresh)

    # Calculate the variance of gaussian noise
    # Note that we should use 3/4 of rho here
    sigma = np.sqrt((2 * clip_thresh * clip_thresh) / (3 * rho / 4))

    noise = np.random.normal(0, sigma, 1)[0]
    actual_sum = np.sum(D)
    estimated_sum = np.sum(D_clipped) + noise

    res = {
        "rho" : rho,
        "beta": beta,
        "tau": tau,
        "n" : D.size,
        "min_val" : min_val,
        "max_val" : max_val,
        "quantile" : m,
        "clip_thresh" : clip_thresh,
        "sigma" : sigma,
        "noise" : noise,
        "actual_sum" : actual_sum,
        "estimated_sum" : estimated_sum,
    }

    return res

In [11]:
def ClippedSumEstimator_laplace(D, min_val, max_val, epsilon, beta):
    # Calculate rank error tau
    tau = GetRankError_laplace(min_val, max_val, 1, epsilon / 4, beta)

    # Get the optimal clipped quantile m
    # Note that we should use 1/4 of epsilon here
    m = GetOptimalQuantile_laplace(D.size, 1, tau, epsilon / 4)

    # Get clipping threshold C
    clip_thresh = PrivQuant_laplace(D, min_val, max_val, m, epsilon / 4)

    # Calculate clipped value of D
    D_clipped = np.clip(D, a_min = -clip_thresh, a_max = clip_thresh)

    # Calculate the variance of gaussian noise
    # Note that we should use 3/4 of tau here
    b = (2 * clip_thresh) / (3 * epsilon / 4)

    noise = np.random.laplace(0, b, 1)[0]
    actual_sum = np.sum(D)
    estimated_sum = np.sum(D_clipped) + noise

    res = {
        "epsilon" : epsilon,
        "beta": beta,
        "tau": tau,
        "n" : D.size,
        "min_val" : min_val,
        "max_val" : max_val,
        "quantile" : m,
        "clip_thresh" : clip_thresh,
        "b" : b,
        "noise" : noise,
        "actual_sum" : actual_sum,
        "estimated_sum" : estimated_sum,
    }

    return res

In [12]:
def SumEstimator_laplace(D, min_val, max_val, epsilon):
    b = (max_val - min_val) / epsilon
    noise = np.random.laplace(0, b)
    actual_sum = np.sum(D)
    estimated_sum = actual_sum + noise

    res = {
        "epsilon" : epsilon,
        "n" : D.size,
        "min_val" : min_val,
        "max_val" : max_val,
        "b" : b,
        "noise" : noise,
        "actual_sum" : actual_sum,
        "estimated_sum" : estimated_sum,
    }

    return res

In [13]:
# Size of dataset
n = 10000

# Largest possible value in dataset
min_val = 0
max_val = 1000

# Generate random content for dataset
D = np.random.randint(min_val, max_val, size=n)

In [14]:
rho = 0.25
beta = 0.001
global_min = 0
global_max = 2147483647
res = ClippedSumEstimator_normal(D, global_min, global_max, rho, beta)
res

{'rho': 0.25,
 'beta': 0.001,
 'tau': 71.62055760518666,
 'n': 10000,
 'min_val': 0,
 'max_val': 2147483647,
 'quantile': 9928.379442394813,
 'clip_thresh': 991,
 'sigma': 3236.592446797506,
 'noise': -313.1285911837273,
 'actual_sum': 5012775,
 'estimated_sum': 5012200.871408816}

In [15]:
epsilon = 0.25
beta = 0.001
global_min = 0
global_max = 2147483647
res = ClippedSumEstimator_laplace(D, global_min, global_max, epsilon, beta)
res

{'epsilon': 0.25,
 'beta': 0.001,
 'tau': 5129.264172263325,
 'n': 10000,
 'min_val': 0,
 'max_val': 2147483647,
 'quantile': 4870.735827736675,
 'clip_thresh': 552,
 'b': 5888.0,
 'noise': 10461.085414098388,
 'actual_sum': 5012775,
 'estimated_sum': 4010341.0854140986}

In [16]:
epsilon = 0.25
beta = 0.001
global_min = 0
global_max = 2147483647
res = SumEstimator_laplace(D, global_min, global_max, epsilon)
res

{'epsilon': 0.25,
 'n': 10000,
 'min_val': 0,
 'max_val': 2147483647,
 'b': 8589934588.0,
 'noise': -8267100424.608283,
 'actual_sum': 5012775,
 'estimated_sum': -8262087649.608283}